In [ ]:
!pip install torchtext==0.15.1
!pip install torch==2.1.0
!pip install transformers==4.27.1
!pip install datasets==2.17.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [ ]:
import torch
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset

qa_dataset = load_dataset('squad', split='train').shard(num_shards=40, index=0)
qa_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 2190
})

In [ ]:
import re

def text_normalize(text):
    text = re.sub(r'[^\w\s]', ' ', text)

    return text

# Define tokenizer function
tokenizer = get_tokenizer('basic_english')

# Create a function to yield list of tokens
def yield_tokens(data):
    for item in data:
        yield tokenizer(
            text_normalize(item['context']) + ' <sep> ' + text_normalize(item['question'])
        )


# Create vocabulary
vocab = build_vocab_from_iterator(
    yield_tokens(qa_dataset),
    specials=['<unk>', '<pad>', '<bos>', '<eos>', '<sep>']
)
vocab.set_default_index(vocab['<unk>'])
vocab.get_stoi()

{'하': 26910,
 '큰': 26909,
 '소': 26904,
 '사람': 26903,
 '나라이름': 26900,
 '魄': 26899,
 '魂': 26898,
 '金陵邑': 26896,
 '越城': 26891,
 '豊臣秀吉': 26890,
 '義皇帝': 26888,
 '禅': 26884,
 '现代汉语通用字表': 26883,
 '浙江': 26879,
 '法王': 26878,
 '汉语水平考试': 26876,
 '水': 26874,
 '校尉': 26873,
 '李閏': 26872,
 '方块字': 26869,
 '平安': 26867,
 '小': 26866,
 '唐入り': 26862,
 '北軍': 26860,
 '凹田': 26859,
 '冶城': 26858,
 '下': 26855,
 'トワイライトプリンセス': 26853,
 'ゼルダの伝説': 26850,
 'ἥλιος': 26849,
 'โรงเร': 26844,
 'สลาม': 26843,
 'ยนศาสนาอ': 26842,
 'ब': 26841,
 'نصراني': 26839,
 'نصارى': 26838,
 'العربية': 26834,
 'языка': 26830,
 'хийума': 26828,
 'стиль': 26826,
 'русского': 26825,
 'рмонтов': 26824,
 'пу': 26823,
 'никола': 26822,
 'насекомое': 26821,
 'н': 26820,
 'мок': 26819,
 'михаи': 26818,
 'кий': 26815,
 'замо': 26811,
 'дов': 26809,
 'голь': 26806,
 'высо': 26804,
 'χαλκός': 26799,
 'φαναῖος': 26797,
 'τεχνικά': 26795,
 'πικρό': 26794,
 'πάνορμος': 26793,
 'λύκη': 26792,
 'λύκειος': 26791,
 'λύκειο': 26790,
 'λόγος': 26789,
 'λυκ

In [ ]:
MAX_SEQ_LEN = 512
PAD_IDX = vocab['<pad>']

def pad_and_truncate(input_ids, max_seq_len):
    if len(input_ids) > max_seq_len:
        input_ids = input_ids[:max_seq_len]
    elif len(input_ids) < max_seq_len:
        input_ids += [PAD_IDX] * (max_seq_len - len(input_ids))

    return input_ids

def vectorize(question, context, answer):
    input_text = text_normalize(question) + ' <sep> ' + text_normalize(context)
    input_ids = [vocab[token] for token in tokenizer(input_text)]
    input_ids = pad_and_truncate(input_ids, MAX_SEQ_LEN)

    answer_ids = [vocab[token] for token in tokenizer(text_normalize(answer))]
    try:
        start_positions = input_ids.index(answer_ids[0])
        end_positions = start_positions + len(answer_ids) - 1
    except:
        start_positions = 0
        end_positions = 0

    input_ids = torch.tensor(input_ids, dtype=torch.long)
    start_positions = torch.tensor(start_positions, dtype=torch.long)
    end_positions = torch.tensor(end_positions, dtype=torch.long)

    return input_ids, start_positions, end_positions

In [ ]:
class QADataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        question_text = item['question']
        context_text = item['context']
        answer_text = item['answers']['text'][0]

        input_ids, start_positions, end_positions = vectorize(question_text, context_text, answer_text)

        return input_ids, start_positions, end_positions

In [ ]:
def decode(input_ids):
    return ' '.join([vocab.lookup_token(token) for token in input_ids])

In [ ]:
for item in qa_dataset:
    question_text = item['question']
    context_text = item['context']
    answer_text = item['answers']['text'][0]

    input_ids, start_positions, end_positions = vectorize(question_text, context_text, answer_text)
    print(input_ids)
    text = decode(input_ids)
    answer_span = input_ids[start_positions:end_positions+1]

    print(text)
    print(decode(answer_span))

    break

tensor([    9,  1255,    47,     5,  1419,   586,  8194,  1477,     8,  4079,
            8,  9275,   243,     4, 10671,     5,   187,    33,    10,   766,
         1325,  5137,     5,   370,   379,    16,  1390,  4217,    12,    10,
         1992,  3679,     6,     5,  1419,   586,  1185,     8,   849,     6,
            5,   370,   379,     7,  6071,    25,    12,    10,  1278,  3679,
            6,  1104,    17,  1433, 26012,    17,     5,  3308, 26126,  1168,
         2108, 22331,   613,     9,     5,   370,   379,    12,     5,  4141,
            6,     5,  2400,  2347,  1185,  1023,     5,  4141,    12,     5,
        12011,    10, 12671,   221,     6,  3965,     7, 23629,    25,    12,
           10, 13474,     6,     5, 12011,    26,  9275,   243,    79,     5,
         1419,   586, 23758,  1375,     9,   644, 16125, 24745,     8,  4079,
           26,     5,   168,     6,     5,   370,  1700,     7,     8,    10,
          583,   281,    18,  5954,   115,   117,  4428,     7, 

In [ ]:
BATCH_SIZE = 128
train_dataset = QADataset(qa_dataset)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
import torch.nn as nn
import torch.optim as optim

class QAModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, n_layers):
        super(QAModel, self).__init__()
        self.input_embedding = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = nn.LSTM(
            embedding_dim, hidden_size,
            num_layers=n_layers,
            batch_first=True,
            bidirectional=True
        )
        self.dropout = nn.Dropout(0.2)

        self.start_linear = nn.Linear(hidden_size * 2, 1)
        self.end_linear = nn.Linear(hidden_size * 2, 1)

    def forward(self, text):
        input_embedded = self.input_embedding(text)
        lstm_out, _ = self.lstm(input_embedded)
        lstm_out = self.dropout(lstm_out)

        start_logits = self.start_linear(lstm_out).squeeze(-1)
        end_logits = self.end_linear(lstm_out).squeeze(-1)

        return start_logits, end_logits

# Model parameters
EMBEDDING_DIM = 64
HIDDEN_SIZE = 128
VOCAB_SIZE = len(vocab)
N_LAYERS = 3
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = QAModel(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_SIZE, N_LAYERS).to(device)

input = torch.randint(0, 10, size=(1, 10)).to(device)
print(input.shape)
model.eval()
with torch.no_grad():
    start_logits, end_logits = model(input)

print(start_logits.shape)

torch.Size([1, 10])
torch.Size([1, 10])


In [ ]:
LR = 1e-2
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

In [ ]:
from tqdm import tqdm
EPOCHS = 10

model.train()
for epoch in tqdm(range(EPOCHS)):
    train_losses = []
    for idx, (input_ids, start_positions, end_positions) in enumerate(train_loader):
        input_ids = input_ids.to(device)
        start_positions = start_positions.to(device)
        end_positions = end_positions.to(device)
        optimizer.zero_grad()
        start_logits, end_logits = model(input_ids)
        start_loss = criterion(start_logits, start_positions)
        end_loss = criterion(end_logits, end_positions)
        total_loss = (start_loss + end_loss) / 2
        total_loss.backward()
        optimizer.step()
        train_losses.append(total_loss.item())
    train_loss = sum(train_losses) / len(train_losses)
    print(f'EPOCH {epoch + 1}\tTraining Loss: {train_loss}')

 10%|█         | 1/10 [00:04<00:40,  4.55s/it]

EPOCH 1	Training Loss: 4.846996837192112


 20%|██        | 2/10 [00:09<00:37,  4.64s/it]

EPOCH 2	Training Loss: 4.318645821677314


 30%|███       | 3/10 [00:16<00:39,  5.63s/it]

...

EPOCH 8	Training Loss: 0.3144798047012753


 90%|█████████ | 9/10 [00:41<00:04,  4.14s/it]

EPOCH 9	Training Loss: 0.1671738645268811


100%|██████████| 10/10 [00:44<00:00,  4.49s/it]

EPOCH 10	Training Loss: 0.09341376875009802


In [ ]:
model.eval()
with torch.no_grad():
    sample = qa_dataset[100]
    context, question, answer = sample['context'], sample['question'], sample['answers']['text'][0]
    # context = 'Jane is a student and she is from Canada'
    # question = 'Where is Jane from?'
    # answer = 'Canada'
    context = text_normalize(context)
    question = text_normalize(question)
    answer = text_normalize(answer)
    input_ids, start_positions, end_positions = vectorize(question, context, answer)
    input_ids = input_ids.to(device)
    start_positions = start_positions.to(device)
    end_positions = end_positions.to(device)
    input_ids = input_ids.unsqueeze(0)
    start_logits, end_logits = model(input_ids)

    offset = len(tokenizer(question)) + 1
    start_position = torch.argmax(start_logits, dim=1).cpu().numpy()[0]
    end_position = torch.argmax(end_logits, dim=1).cpu().numpy()[0]

    start_position -= offset
    end_position -= offset

    start_position = max(start_position, 0)
    end_position = min(end_position, len(tokenizer(context)) - 1)

    if end_position >= start_position:
        # Extract the predicted answer span
        context_tokens = tokenizer(context)
        predicted_answer_tokens = context_tokens[start_position:end_position + 1]
        predicted_answer = ' '.join(predicted_answer_tokens)
    else:
        predicted_answer = ''

    print(f'Context: {context}')
    print(f'Question: {question}')
    print(f'Start position: {start_position}')
    print(f'End position: {end_position}')
    print(f'Answer span: {predicted_answer}')
    print(f'Label: {answer}')

Context: The city and surrounding area suffered the bulk of the economic damage and largest loss of human life in the aftermath of the September 11  2001 attacks when 10 of the 19 terrorists associated with Al Qaeda piloted American Airlines Flight 11 into the North Tower of the World Trade Center and United Airlines Flight 175 into the South Tower of the World Trade Center  and later destroyed them  killing 2 192 civilians  343 firefighters  and 71 law enforcement officers who were in the towers and in the surrounding area  The rebuilding of the area  has created a new One World Trade Center  and a 9 11 memorial and museum along with other new buildings and infrastructure  The World Trade Center PATH station  which opened on July 19  1909 as the Hudson Terminal  was also destroyed in the attack  A temporary station was built and opened on November 23  2003  A permanent station  the World Trade Center Transportation Hub  is currently under construction  The new One World Trade Center i